In [ ]:
import partitura as pt
import numpy as np

In [ ]:
score = pt.load_musicxml('/Users/johan/Documents/MuseScore4/Scores/ChordTest_From_Yamaha_Chord_Guide.musicxml')

In [3]:
part = score.parts[0]

### Read everything but morphetic pitch straight from score part

In [10]:
staff_number = 0
notes = []
for note_or_staff in part.iter_all():
    if isinstance(note_or_staff, pt.score.Staff):
        staff_number = note_or_staff.number
    elif isinstance(note_or_staff, pt.score.Note):
        start_time = note_or_staff.start.t
        end_time = note_or_staff.end_tied.t
        notes.append((part.quarter_map(start_time), note_or_staff.midi_pitch, part.quarter_map(note_or_staff.duration_tied), staff_number, part.measure_number_map(start_time)))
note_array = np.array(notes, dtype=[('onset', 'f4'), ('midi_pitch', 'i4'), ('duration', 'f4'), ('staff', 'i4'), ('measure', 'i4')])

### Compute morphetic pitch and add it to note array

In [11]:
from partitura.musicanalysis.pitch_spelling import compute_morphetic_pitch, compute_morph_array, compute_chroma_array, compute_chroma_vector_array, chromatic_pitch_from_midi

def morphetic_pitch_array(note_array, K_pre=10, K_post=40):
    # Adapted from partitura.musicanalysis.pitch_spelling.ps13s1
    pitch_sort_idx = note_array['midi_pitch'].argsort()
    onset_sort_idx = np.argsort(note_array[pitch_sort_idx]['onset'], kind='mergesort')
    sort_idx = pitch_sort_idx[onset_sort_idx]

    sorted_ocp = np.column_stack(
        (
            note_array[sort_idx]['onset'],
            chromatic_pitch_from_midi(note_array[sort_idx]['midi_pitch']),
        )
    )

    chroma_array = compute_chroma_array(sorted_ocp=sorted_ocp)
    chroma_vector_array = compute_chroma_vector_array(chroma_array, K_pre, K_post)
    morph_array = compute_morph_array(chroma_array, chroma_vector_array)

    return compute_morphetic_pitch(sorted_ocp, morph_array)

In [18]:
morphetic_pitches = np.array(morphetic_pitch_array(note_array), dtype=[('morphetic_pitch', 'i4')])
all_note_info = np.lib.recfunctions.merge_arrays((note_array, morphetic_pitches), flatten=True)

### Write note array to csv

In [ ]:
all_note_info['onset']

In [ ]:
all_note_info['midi_pitch']

In [ ]:
all_note_info['morphetic_pitch']

In [ ]:
all_note_info['duration']

In [ ]:
all_note_info['staff']

In [ ]:
all_note_info['measure']

In [16]:
# exercise for the reader